In [1]:
import pandas as pd
import numpy as np

In [2]:
abm = pd.read_csv("../Data/account_based_measures.csv")
abm.head()

,user_name,Twitter Account Duration(till 26/1/2020),Avg. Tweet Rate(per Month),Avg. Retweet Rate(per Month),Avg. Reply Rate(per Month),Total Normal Tweets,Total Retweets,Total Replies,Total Quotes,Avg. Fav Count(per Month),Avg. Retweeted Count(per Month),Avg. Quote Count(per Month)
0,001Danish,328285108.0,4.300,7.745,2.435,215,364,56,188,225.533,37526.150,4.947
1,0thFromOurStar,55209782.0,9.846,3.308,21.529,128,43,366,37,3501.391,1295.696,3.364
2,0x0000EBFE,223236506.0,2.087,5.988,1.719,48,503,55,4,5.386,1159.159,1.000
3,0xbharath,289685006.0,5.493,5.349,2.974,412,337,116,25,38.348,1252.663,1.667
4,0xdekster,181467531.0,7.057,9.000,14.688,247,243,470,71,119.452,11512.976,3.227


In [3]:
abf = pd.read_csv("../Data/account_based_features.csv")
abf.head()

,User ID,User Name,Followers,Following,Follower/Following Ratio,Total Status,Retweet Count,Total Favourites
0,2615733620,1000DegreesX,148,136,1.088,504,44,109
1,437763875,100Security,949,2856,0.332,549,2,372
2,269513390,121giving,3205,3286,0.975,617,78,95
3,3005258262,138_das,69,42,1.643,618,292,296
4,2160172507,13kamilkhan,3518,172,20.453,704,176,2738


In [4]:
ndays = 30
dt = 1/24 #time step in days
npts = int(ndays/dt)
N = 90000


def get_score(username):
    follower_count = abf[abf['User Name'] == username]['Followers'].values[0]
    tweet_rate = abm[abm['user_name'] == username]['Avg. Tweet Rate(per Month)'].values[0] / 30
    retweet_rate = abm[abm['user_name'] == username]['Avg. Retweet Rate(per Month)'].values[0] / 30
    
    S = np.zeros(npts) #susceptible
    I = np.zeros(npts) #infective   
    beta = 1/follower_count #infection rate
    gamma = (1.0/14.0) #recovery rate

    I[0] = follower_count #initial infective proportion
    S[0] = N - I[0] #initial susceptible

    for i in range(npts-1):
        ds = int(S[i] - beta*(S[i]*I[i])*dt)
        di = int(I[i] + (beta*S[i]*I[i]-gamma*I[i])*dt)
        if di >= N:
            S[i+1] = 0
            I[i+1] = N
            break
        else:
            S[i+1] = ds
            I[i+1] = di
    
#     calculating score
    t = (np.where(I == I.max())[0][0] + 1) * dt  
    time_per_user = t / I.max()
    rem = 1 if N - I.max() <= 0 else N - I.max()
    
    score = t + (rem * time_per_user)
    return score*24

In [5]:
import csv

usernames = abf['User Name'].tolist()
print(len(usernames))
for i in usernames:
    if i not in abm['user_name'].tolist():
        usernames.remove(i)
print(len(usernames))

with open('./sir_v2.csv', 'w', newline='') as csvfile:
    fieldnames = ['username', 'score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    count = 0
    for i in usernames:
        try:
            score = get_score(i)
            print(i, "-->" , score)
            count += 1
            writer.writerow({'username': i, 'score': score})

#             if count == 5:
#                 break
        except Exception as e:
            print(e)   

8296
8296
1000DegreesX --> 3.0000333333333336
100Security --> 5.000055555555555
121giving --> 7.05194936028745
138_das --> 3.0000333333333336
13kamilkhan --> 8.062258552152734
13pt --> 17.5856004965346
14Swathi --> 5.000055555555555
18Bodaji --> 4.000044444444445
19thCenturyShop --> 5.000055555555555
19thmilecapsol --> 4.000044444444445
207teacherguy --> 5.000055555555555
23septemberrrr --> 12.242954633051443
24namratha --> 4.000044444444445
321Foundation --> 5.000055555555555
40MILLIONBUCKS --> 30.01715265866209
4WeddingsOnHulu --> 8.062529394638418
4WorkerRights --> 8.078903962029152
501NotOut --> 5.000055555555555
5aelo --> 33.586940819569506
5h1vang --> 4.000044444444445
5onyaHorton --> 4.000044444444445
5SmL9e5kpyuincH --> 5.000055555555555
609402 --> 7.000077777777777
60_decibels --> 5.000055555555555
62f60ca138454a3 --> 3.0000333333333336
63Jbk14zFFjBg4O --> 4.000044444444445
6thFromOurStar --> 19.793041183415514
777kabra --> 3.0000333333333336
90sWWE --> 1.0000020870596042
9Har

<ipython-input-4-b950e59e358f>:14: RuntimeWarning: divide by zero encountered in longlong_scalars
  beta = 1/follower_count #infection rate
<ipython-input-4-b950e59e358f>:21: RuntimeWarning: invalid value encountered in double_scalars
  ds = int(S[i] - beta*(S[i]*I[i])*dt)


AanchalBajpai --> 4.000044444444445
aankjain --> 4.000044444444445
AAPNDGupta --> 12.242954633051443
AapRaghu --> 4.000044444444445
AAP_ki_BAAT --> 3.0000333333333336
aaqibkhan678 --> 3.0000333333333336
aaranrpatel --> 4.000044444444445
aaronaders --> 4.000044444444445
aarondemello --> 4.000044444444445
AaronHadlow --> 17.631400025352338
AaronLennon12 --> 1.0000016531603466
aaronportnoy --> 27.671022290545736
AarthyRao --> 3.0000333333333336
aashim90 --> 11.189222179524853
Aashima_Prasad --> 4.000044444444445
Aashish0506 --> 3.0000333333333336
aashish15choud1 --> 3.0000333333333336
Aashish_Mangal1 --> 4.000044444444445
Aashish_omveer --> 4.000044444444445
AashtiZaidiHai --> 4.000044444444445
AATSJournals --> 12.249897917517353
aayushbansal --> 4.000044444444445
AA_speaks --> 4.000044444444445
abaoker --> 4.000044444444445
abarisiddiqui --> 17.600368112274243
abbirdin --> 26.570305027101707
abbynissenbaum --> 11.18732555117354
abdoul_k --> 5.000055555555555
abehmard --> 5.00005555555555

Aishwarya_R_M --> 4.000044444444445
aishwin --> 6.000066666666667
aitcpriyanka --> 15.436157197251221
aitcsudip --> 27.220142905750254
AITC_Parliament --> 24.237456823370994
aitmc_arup --> 5.000055555555555
AiyarYamini --> 18.680165584677653
ajaykumar5571 --> 3.0000333333333336
ajaypatnaik --> 3.0000333333333336
AjayPrabhuQuEST --> 4.000044444444445
ajayshr --> 5.000055555555555
ajay_kanchan --> 5.000055555555555
AjeetYadav1984 --> 3.0000333333333336
ajendrasingh81 --> 3.0000333333333336
AJFiore --> 15.426984653007121
ajinkya78 --> 4.000044444444445
ajinkyaa_k --> 5.000055555555555
Ajinkya_Jhingi --> 4.000044444444445
Ajitessh --> 4.000044444444445
AjithKarimpana --> 5.000055555555555
AjRossdohan --> 4.000044444444445
ajstone24 --> 5.000055555555555
ajubhat --> 4.000044444444445
ak3754 --> 3.0000333333333336
AkaManx --> 3.0000333333333336
AKAne_GUcchi66 --> 38.05643115245637
akanksha_bapna --> 4.000044444444445
AKASHAworld --> 22.0143734057051
akashjain --> 4.000044444444445
akashkinar

AngelaStent --> 18.69288285793409
angelinaksun --> 3.0000333333333336
AngeloCassisa --> 5.000055555555555
ANGELZOFFICIAL --> 28.863628264975233
AngilaLumumba --> 4.000044444444445
angiperera --> 46.61150761068654
angira_biswas --> 4.000044444444445
angira_dhar --> 24.242850115944066
AngryChefASMR --> 5.000055555555555
anichakladar --> 9.101634923310298
anideshpandelab --> 4.000044444444445
aniketdivekar --> 7.000077777777777
anikethdesai --> 3.0000333333333336
anikethjain --> 4.000044444444445
Aniketistic --> 4.000044444444445
anikgoel --> 10.164438019515721
anilbehl54 --> 4.000044444444445
anilmdave --> 32.46089664210058
anilpatni --> 4.000044444444445
AnilShri --> 7.000077777777777
AnilsinghHisua --> 5.000055555555555
anilsonadi --> 11.194536161744086
AnilTanwar --> 22.041073365287062
AnirbanM16 --> 3.0000333333333336
anirbansparsha --> 3.0000333333333336
AnirbanSpeaketh --> 48.37347393207238
aniruddhapant --> 4.000044444444445
aniruddha_verma --> 4.000044444444445
AnirudhHcps --> 4.

...

SoodVandna --> 4.000044444444445
soorajbhalla --> 4.000044444444445
sooraj_kamath --> 3.0000333333333336
sooriofficial --> 1.0000004936808846
soosairajmichal --> 15.431216780019433
SopanDeb --> 1.0000057049947229
sophdevine77 --> 29.983937176512576
SophieJusticeW --> 4.000044444444445
SOPMedUSA --> 4.000044444444445
SoprAboist --> 4.000044444444445
SorayyaKhan --> 4.000044444444445
sortizreina --> 4.000044444444445
souadmassioff --> 9.106853750674581
Soulfulsachin --> 10.141416417826356
SoumyaMukerji_ --> 4.000044444444445
SoumyaShetty25 --> 4.000044444444445
soumyavajpayee --> 4.000044444444445
Soundarya_J --> 4.000044444444445
souravact --> 7.000077777777777
Souravi8 --> 3.0000333333333336
souravsarkar03 --> 4.000044444444445
sourendrospeaks --> 5.000055555555555
sourish_m --> 4.000044444444445
SouthAsiaLW --> 3.0000333333333336
SouthernHReview --> 13.300139821982743
SouthwordState --> 4.000044444444445
SouvikBhattac19 --> 3.0000333333333336
space4ramzan --> 3.0000333333333336
SpaceF

SuneLuus --> 25.40459164471208
suneshgovind --> 4.000044444444445
SunflowerTherRC --> 7.000077777777777
sung1e_thakur --> 4.000044444444445
SunilASDAA --> 5.000055555555555
sunildang --> 4.000044444444445
sunildst --> 4.000044444444445
sunilpandey0 --> 4.000044444444445
SunilSGangurde --> 4.000044444444445
Sunitavenugopal --> 4.000044444444445
sunittaradia --> 3.0000333333333336
sunnykaushal89 --> 13.329232031169894
Sunnysaini_spks --> 4.000044444444445
sunnysarochbjym --> 4.000044444444445
sunny_kumar0705 --> 4.000044444444445
Sunriseclick --> 4.000044444444445
suntroncorp --> 6.000066666666667
sunummm --> 5.000055555555555
Suparna_Aangan --> 4.000044444444445
SuperTalkTweets --> 3.0000333333333336
super_tech007 --> 3.0000333333333336
suppcoolgayatri --> 4.000044444444445
surajdidwania --> 4.000044444444445
surajeet_ghosh --> 4.000044444444445
surajkmurthy --> 3.0000333333333336
SureshGorakala --> 4.000044444444445
surfsquire --> 4.000044444444445
surgeonwhocodes --> 3.000033333333333

TheNiteshAnand --> 4.000044444444445
theowalcott --> 1.0000004383000618
ThePeoplesSay --> 25.42522511918074
ThePeteOKeefe --> 4.000044444444445
ThePiyushOfcl --> 3.0000333333333336
ThePunSmith --> 4.000044444444445
therascalas --> 5.000055555555555
TheRealAnshh --> 3.0000333333333336
therealomofuji --> 8.068221293380695
therealtabrett --> 16.50959620279287
thereasaf --> 4.000044444444445
TheRicardoAlvez --> 13.304677105721009
TheRootAcademy --> 10.141302143195185
thesense360 --> 6.000066666666667
theshiftseries --> 4.000044444444445
theshrutster --> 5.000055555555555
TheSillyPoint_ --> 4.000044444444445
TheSilverLab --> 5.000055555555555
thesoul_traders --> 5.000055555555555
TheStudyCastle --> 4.000044444444445
thesulk --> 1.000001876623279
TheTanayJain --> 4.000044444444445
TheTannedDon --> 4.000044444444445
theterenceyung --> 5.000055555555555
ThethBiharan --> 4.000044444444445
thetiehub --> 4.000044444444445
TheTrickRoom --> 5.000055555555555
thetrippytoes --> 4.000044444444445
thev

Valeriasketches --> 4.000044444444445
vamsichada --> 3.0000333333333336
vamsikrshna --> 3.0000333333333336
Vandersay --> 38.58355483151848
VanGoghDaily --> 12.266036707250592
VaniLach7iQUEEN --> 3.0000333333333336
vanimt --> 4.000044444444445
vanishiv_ --> 4.000044444444445
vanky_arora --> 6.000066666666667
VanzaraDg --> 45.23238135923306
Varadha90064366 --> 4.000044444444445
varunbadola7 --> 6.000066666666667
VarunDrive --> 3.0000333333333336
varungujjar --> 3.0000333333333336
varunstweets --> 5.000055555555555
vasanthsarathy --> 4.000044444444445
vashubhagnani --> 18.723345237682466
vatsalapm --> 3.0000333333333336
vaznath --> 4.000044444444445
vchalkindia --> 4.000044444444445
VC_NALSAR --> 41.0573816980248
vdavson1 --> 3.0000333333333336
VecchioneAuthor --> 4.000044444444445
VEDAKOLKATA --> 4.000044444444445
veddev4 --> 3.0000333333333336
vedhalam_maman --> 6.000066666666667
veenu1981 --> 4.000044444444445
veermarathi --> 10.14267360876327
veersarya --> 6.000066666666667
veer --> 8

Wild_Fights --> 18.984985730239483
willbrac --> 4.000044444444445
WillHitchUVA --> 12.249203234697003
William28538949 --> 5.000055555555555
WilliamMattull --> 4.000044444444445
william_keely --> 4.000044444444445
WilmotsMarc --> 1.0000032710524938
winadiwiratama --> 5.000055555555555
Windiesman --> 3.0000333333333336
WindrushStrikes --> 4.000044444444445
WindSCMPhD --> 26.544287669293567
Wine_Raconteur --> 14.360610895828582
winni_bill --> 7.000077777777777
wiomax --> 21.987482258777543
wiredspacephoto --> 16.529494817314646
WisataIndo99 --> 4.000044444444445
wisebuyindia --> 4.000044444444445
WittedNote --> 13.293490734323338
WitthoeftCarina --> 45.25683252457697
WIT_Girls --> 4.000044444444445
wjbili --> 4.000044444444445
WKulawansha --> 5.000055555555555
wlopwangling --> 1.000001859316664
WMKleinMD --> 6.000066666666667
WMowla --> 4.000044444444445
WOAHmaniya --> 3.0000333333333336
WolfsSharon --> 5.000055555555555
wolftoneHQ --> 4.000044444444445
woman_rednow --> 4.000044444444445
